## Rename Oddity Files to Standard Format (.ps1)

VERSION: `Rename_Oddity`

### Advanced Explanation

This PowerShell script begins by setting `$currentDirectory` to the current working directory using `Get-Location`. It then retrieves `.txt` files containing 'cat' in their names from `$currentDirectory` using `Get-ChildItem`. Iterating through these files with a `foreach` loop, the script reads each file's content into `$content`.

For each file, the script searches `$content` for a line matching a date pattern ("SessionDate: DD-MM-YYYY") using `Where-Object` and a regex pattern. If such a line is found, the date is extracted and reformatted into "YYYYDDMM" format using regex groups.

Next, it checks if the file name matches the pattern "cat.*_v2-#", extracting the participant ID using regex. The script then constructs a new base name for the file, combining the participant ID and the formatted date, and renames the `.txt` file accordingly.

Additionally, the script attempts to find a corresponding `.edat2` file with the same base name as the `.txt` file. If found, this file is also renamed using the new base name format. Renaming operations and their results are outputted using `Write-Output`.

### User-Friendly Explanation

This script is designed to rename specific text files and related files in a folder on your computer. It looks for text files that include 'cat' in their name and have a specific date format inside ("SessionDate: DD-MM-YYYY"). When such a file is found, the script changes its name to include a participant ID (extracted from the original file name) and the date from inside the file, but in a different format (YYYYDDMM).

The script also looks for related files with the extension '.edat2' that have the same original name as the text file. If such a file is found, it is renamed to match the new name of the text file. After renaming each file, the script reports the old and new names. This process helps in organizing and identifying files based on specific data contained within them.

### Oversimplified Explantion

Renames raw Oddity files to a format like 123_20231216 (123 being the ID).  Since it reads the txt for the date, there is no need to rename on the same day. 

In [ ]:
$currentDirectory = Get-Location
$txtFiles = Get-ChildItem -Path $currentDirectory\*cat*.txt

foreach ($txtFile in $txtFiles) {
    $content = Get-Content $txtFile.FullName
    $dateLine = $content | Where-Object { $_ -match "SessionDate: (\d{2})-(\d{2})-(\d{4})" }

    if ($dateLine) {
        $dateMatches = [regex]::Match($dateLine, "(\d{2})-(\d{2})-(\d{4})")
        $formattedDate = "{0}{1}{2}" -f $dateMatches.Groups[3], $dateMatches.Groups[1], $dateMatches.Groups[2]

        if ($txtFile.Name -match "cat.*_v2-(\d+)-1") {
            $participantID = $matches[1]
            $baseNameWithoutExtension = $txtFile.BaseName
            $newBaseName = "${participantID}_${formattedDate}"

            $ftxt = "$newBaseName.txt"
            Rename-Item -Path $txtFile.FullName -NewName $ftxt
            Write-Output "Renamed $($txtFile.Name) to $ftxt"

            $edat2FileName = $baseNameWithoutExtension + ".edat2"
            $edat2File = Get-ChildItem -Path "$currentDirectory\$edat2FileName" -ErrorAction SilentlyContinue
            if ($edat2File) {
                $fedat = "$newBaseName.edat2"
                Rename-Item -Path $edat2File.FullName -NewName $fedat
                Write-Output "Renamed $($edat2File.Name) to $fedat"
            }
        }
    }
}


VERSION: `Rename_Oddity_Today`

This assumes the date so it has to be run on the same day

In [ ]:
$currentDate = Get-Date -Format "yyyyMMdd"

Get-ChildItem -Path .\*cat* | ForEach-Object {

    if ($_.Name -match "_v2-(\d+)-1") {

        $participantID = $matches[1]
        $newFileName = "${participantID}_$currentDate$($_.Extension)"

        Rename-Item -Path $_.FullName -NewName $newFileName
        Write-Output "Renamed $($_.Name) to $newFileName"
    }
}
